In [1]:
import pandas as pd
import gensim.corpora as corpora
import gensim

/Users/mmilosh/Library/Python/3.8/lib/python/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
pkl_path = 'sampled_processed_extant_posts_june_july.pkl'
posts_df = pd.read_pickle(pkl_path)
posts_df.head()
# need to delete 'BotForceOne', 'AutoModerator'!

,orig_selftext,has_long_token,text_clean_space,text_clean_punc_lower,len_clean,tokens,tokens_clean,tokens_lemma,bigrams,trigrams
2490481,Even if they hate Trump they could at least ac...,False,Even if they hate Trump they could at least ac...,even if they hate trump they could at least ac...,139,"[even, if, they, hate, trump, they, could, at,...","[even, hate, trump, could, least, acknowledge,...","[even, hate, trump, could, least, acknowledge,...","[(even, hate), (hate, trump), (trump, could), ...","[(even, hate, trump), (hate, trump, could), (t..."
2490482,Heterosexual reproduction is a mystery to admi...,False,Heterosexual reproduction is a mystery to admi...,heterosexual reproduction is a mystery to admi...,76,"[heterosexual, reproduction, is, a, mystery, t...","[heterosexual, reproduction, mystery, admins, ...","[heterosexual, reproduction, mystery, admins, ...","[(heterosexual, reproduction), (reproduction, ...","[(heterosexual, reproduction, mystery), (repro..."
2490483,Were you expecting the actual pope?,False,Were you expecting the actual pope?,were you expecting the actual pope,34,"[were, you, expecting, the, actual, pope]","[expecting, actual, pope]","[expecting, actual, pope]","[(expecting, actual), (actual, pope)]","[(expecting, actual, pope)]"
2490484,"Be fruitful and multiply, not fruity and blow ...",False,"Be fruitful and multiply, not fruity and blow ...",be fruitful and multiply not fruity and blow a...,50,"[be, fruitful, and, multiply, not, fruity, and...","[fruitful, multiply, fruity, blow, guy]","[fruitful, multiply, fruity, blow, guy]","[(fruitful, multiply), (multiply, fruity), (fr...","[(fruitful, multiply, fruity), (multiply, frui..."
2490485,The term has completely lost all meaning. It's...,False,The term has completely lost all meaning. It's...,the term has completely lost all meaning its j...,104,"[the, term, has, completely, lost, all, meanin...","[term, completely, lost, meaning, generalizati...","[term, completely, lost, meaning, generalizati...","[(term, completely), (completely, lost), (lost...","[(term, completely, lost), (completely, lost, ..."


In [6]:
tokens = posts_df['tokens_lemma']
del posts_df

id2word = corpora.Dictionary(tokens)
corpus = [id2word.doc2bow(text) for text in tokens]

In [7]:
def load_models():
    trained_models = dict()
    for num_topics in range(5, 30, 2):
        model_path = path_to_box + 'lda_models/lda_' + str(num_topics) + '_topics'
        print("Loading LDA(k=%d) from %s" % (num_topics, model_path))
        trained_models[num_topics] = gensim.models.LdaMulticore.load(model_path)
    return trained_models

In [9]:
# Write models
# Won't keep results in memory

path_to_box = '' # to RedditQuarantineNLP
for num_topics in range(5, 30, 2):
    print("k = " + str(num_topics))
    lda = gensim.models.LdaMulticore(corpus = corpus, id2word = id2word,
                                     num_topics = num_topics, workers = 3, # use more if you can (#cores - 1)
                                     iterations=100, random_state=0, eval_every=None)
    model_path = path_to_box + 'lda_models/lda_' + str(num_topics) + '_topics'
    lda.save(model_path, separately = False)

In [56]:
# Load results
trained_models = dict()
for num_topics in range(5, 30, 2):
    model_path = path_to_box + 'lda_models/lda_' + str(num_topics) + '_topics'
    print("Loading LDA from %s" % (model_path))
    trained_models[num_topics] = gensim.models.LdaMulticore.load(model_path)

# Coherence
cm = gensim.models.CoherenceModel.for_models(
    trained_models.values(), id2word, texts = tokens, coherence = 'c_v')

coherence_estimates = cm.compare_models(trained_models.values())
coherences = dict(zip(trained_models.keys(), coherence_estimates))

def print_coherence_rankings(coherences):
    avg_coherence = \
        [(num_topics, avg_coherence)
         for num_topics, (_, avg_coherence) in coherences.items()]
    ranked = sorted(avg_coherence, key=lambda tup: tup[1], reverse=True)
    print("Ranked by average '%s' coherence:\n" % cm.coherence)
    for item in ranked:
        print("num_topics=%d:\t%.4f" % item)
    print("\nBest: %d" % ranked[0][0])
    
print_coherence_rankings(coherences)

Ranked by average 'c_v' coherence:

num_topics=7:	0.6971
num_topics=9:	0.6918
num_topics=5:	0.6888
num_topics=13:	0.6794
num_topics=11:	0.6667
num_topics=15:	0.6618
num_topics=17:	0.6452
num_topics=19:	0.6381
num_topics=23:	0.6373
num_topics=27:	0.6319
num_topics=21:	0.6254
num_topics=29:	0.6039
num_topics=25:	0.5975

Best: 7


In [73]:
trained_models[25].print_topics()

[(7,
  '0.048*"woman" + 0.048*"white" + 0.036*"gay" + 0.030*"black" + 0.024*"men" + 0.024*"people" + 0.023*"hate" + 0.022*"racist" + 0.017*"muslim" + 0.014*"dude"'),
 (8,
  '0.056*"automatically" + 0.045*"trump" + 0.026*"president" + 0.021*"watch" + 0.013*"submission" + 0.013*"week" + 0.013*"youtube" + 0.011*"time" + 0.010*"clinton" + 0.010*"donald"'),
 (0,
  '0.064*"good" + 0.025*"make" + 0.018*"look" + 0.017*"win" + 0.014*"trump" + 0.013*"sad" + 0.012*"really" + 0.011*"sense" + 0.010*"wow" + 0.010*"hope"'),
 (1,
  '0.026*"u" + 0.022*"country" + 0.022*"child" + 0.020*"kid" + 0.015*"family" + 0.014*"law" + 0.012*"police" + 0.010*"home" + 0.010*"parent" + 0.010*"back"'),
 (4,
  '0.027*"right" + 0.013*"people" + 0.013*"free" + 0.011*"rule" + 0.011*"want" + 0.009*"speech" + 0.008*"community" + 0.007*"school" + 0.007*"law" + 0.007*"place"'),
 (2,
  '0.026*"read" + 0.017*"baby" + 0.013*"biden" + 0.011*"article" + 0.011*"story" + 0.011*"kill" + 0.010*"hear" + 0.010*"joe" + 0.009*"one" + 0.00